In [ ]:
import pyarrow
import pyarrow.parquet as pq
from pathlib import Path
import pandas as pd

In [ ]:
user_data_dir = Path("/Volumes/One Touch/Downloads/files")
all_user_data = list(user_data_dir.glob("0*.parquet"))
ny_position_data_file = Path("ny_positions.csv")
result_dir = Path("filtered_user_data")
result_dir.mkdir(exist_ok=True)

In [ ]:
ny_position_data = pd.read_csv(ny_position_data_file)
ny_position_data["user_id"] = ny_position_data["user_id"].astype(int).astype(str)
ny_position_data.drop_duplicates(subset=["user_id"], inplace=True)

In [ ]:
# show how many row groups are in the parquet file
for parquet_file in all_user_data:
    output_file = result_dir / parquet_file.name.replace(".parquet", ".csv")
    if output_file.exists():
        continue
    print("processing ", parquet_file)
    data = pq.ParquetFile(parquet_file)
    n_row_groups = data.num_row_groups
    all_merged_data = []
    for ri in range(n_row_groups):
        if ri % 20 == 0:
            print("processing row group ", ri, " out of ", n_row_groups)
        table = data.read_row_group(ri).to_pandas()
        table["user_id"] = table["user_id"].astype(str)
        new_marged_table = pd.merge(table, ny_position_data, on="user_id", how="inner")
        all_merged_data.append(new_marged_table)
    print(len(all_merged_data))
    all_merged_data = pd.concat(all_merged_data)
    all_merged_data.to_csv(result_dir / parquet_file.name.replace(".parquet", ".csv"), index=False)
        
        